## General Colab Tips
- Modify files by opening/editing them in the UI (double-click to open).
- `Right click > Refresh` in the Colab file explorer to update the directory.
- All files are lost when the Colab session disconnects, so make sure back up your work.
- Do **not** use `drive.mount` for your datasets! Reading from GDrive is super slow.
- Instead, place datasets into the `/content/` folder and modify your data accordingly.

**Make a copy of this notebook and modify this to whatever workflow you prefer!**

If you have some additional colab tips, please share them on the discussion forum.

## Setup

First, enable a GPU runtime via `Runtime > Change runtime type > T4 GPU`

Next, upload the your project files to the Colab. You can do this by either
- using Github (**recommended**)
- uploading files manually using the UI

## Github Setup

You can use git from within Google Colab!

For this section, we assume you know how to use git and have already pushed the starter code to a private repo.

Before you continue, make sure you download and push the starter code to your repo.  
It's a good idea to structure your repo something like
```
online_deep_learning/
    homework1/
    homework2/
    ...
```

We highly recommend using this workflow as you'll be able to easily pull/commit your changes after modifying your model on Colab.

To do this, you'll need a personal access token from [https://github.com/settings/tokens](https://github.com/settings/tokens)

The easiest thing to do is select "classic" token and make sure you have the `repo` scope selected to allow access to your private repos.
There's also fine-grained tokens where you can select access to specific repos.

Once you have your token, fill in your information and then run the following cell to clone your git repo to the Colab instance.


In [ ]:
import os

# Fill in your GitHub credentials
os.environ['USER'] = 'YOUR_GITHUB_USERNAME'
os.environ['REPO'] = 'YOUR_REPO_NAME'
os.environ['TOKEN'] = 'YOUR_GITHUB_TOKEN'

# do everything in colab's "root" directory
%cd /content
!git clone https://${TOKEN}@github.com/${USER}/${REPO}.git

# make sure your repo shows up
%ls


## Code Setup

Move into `homework5/` so we can continue setting up the data / code for training.

This will be the main working directory and the training/grading must be run from this directory.


In [ ]:
# navigate to your repo
%cd /content/{os.environ['REPO']}
%ls

# go to a specific homework
%cd homework5
%ls


## Install Dependencies

Install the required packages for Homework 5.


In [ ]:
# Install PyTorch (Colab usually has it, but this ensures latest version)
# Check https://pytorch.org/get-started/locally/ for the latest command
# For Colab, CUDA is usually available, so we install CUDA version
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies
%pip install -r requirements.txt


## Dataset Setup

Now that your code is all ready, the next step is to download the datasets.

Note: it's good practice to add data directories like `data/` to your `.gitignore` so you don't accidentally commit them to your repo.

Since the datasets used in this class are relatively small, we can simply re-download them if the compute instance crashes/restarts.


In [ ]:
# Download and extract the SuperTuxKart image dataset
!wget https://utexas.box.com/shared/static/qubjm5isldqvyimfj9rsmbnvnbezwcv4.zip -O supertux_data.zip
!unzip -q supertux_data.zip -d data/

# Verify the data structure
%ls data/


## Model Implementation + Training

Now you should be all set up.
Next, you'll need to implement:
- `homework/ae.py` - Patch-level auto-encoder
- `homework/bsq.py` - Binary Spherical Quantization
- `homework/autoregressive.py` - Auto-regressive model
- `homework/generation.py` - Generation function (already provided)

And then you're ready to train!


In [ ]:
# Pull latest changes from your repo
!git pull origin main


In [ ]:
# Train PatchAutoEncoder (Part 1)
# This trains a patch-level auto-encoder
!python -m homework.train PatchAutoEncoder --epochs 5 --batch_size 64


In [ ]:
# Train BSQPatchAutoEncoder (Part 2)
# This combines your auto-encoder with Binary Spherical Quantization
# Make sure to use patch_size=5 and codebook_bits=10 for compatibility
!python -m homework.train BSQPatchAutoEncoder --epochs 5 --batch_size 64


## Tokenization

After training the BSQPatchAutoEncoder, we need to tokenize the training and validation datasets.

**Important:** Replace `YOUR_BSQPatchAutoEncoder.pth` with the actual checkpoint filename from the `checkpoints/` directory.


In [ ]:
# List available checkpoints to find your BSQPatchAutoEncoder
!ls -lh checkpoints/ | grep BSQPatchAutoEncoder


In [ ]:
# Tokenize the training set
# Replace the checkpoint path with your actual checkpoint filename
!python -m homework.tokenize checkpoints/YOUR_BSQPatchAutoEncoder.pth data/tokenized_train.pth data/train/

# Tokenize the validation set
!python -m homework.tokenize checkpoints/YOUR_BSQPatchAutoEncoder.pth data/tokenized_valid.pth data/valid/


In [ ]:
# Check the size of tokenized datasets
!du -hs data/tokenized_train.pth data/tokenized_valid.pth


In [ ]:
# Train AutoregressiveModel (Part 3)
# This trains the auto-regressive transformer on tokenized images
!python -m homework.train AutoregressiveModel --epochs 5 --batch_size 64


## Generation (Part 4)

Generate samples from your trained models.

**Important:** Replace the checkpoint paths with your actual checkpoint filenames.


In [ ]:
# List available checkpoints
!ls -lh checkpoints/ | tail -10


In [ ]:
# Generate images
# Replace checkpoint paths with your actual filenames
# Format: tokenizer_checkpoint autoregressive_checkpoint num_images output_dir
!python -m homework.generation checkpoints/YOUR_BSQPatchAutoEncoder.pth checkpoints/YOUR_AutoregressiveModel.pth 10 generations/


In [ ]:
# Display generated images
from IPython.display import Image, display
from pathlib import Path

generation_dir = Path("generations")
if generation_dir.exists():
    for img_path in sorted(generation_dir.glob("*.png")):
        print(img_path)
        display(Image(str(img_path)))


## Grader

Run the following cell to grade your homework.


In [ ]:
# Make sure you're in the homework5 directory
%cd /content/{os.environ['REPO']}/homework5

# Run the grader
!python3 -m grader homework -vv --disable_color


## Update your changes

Commit and push your code changes to GitHub.


In [ ]:
%cd /content/{os.environ['REPO']}/homework5
%ls
!git status

# Be careful not to "git add *" since there are datasets and logs
!git add homework/*.py
!git config --global user.email "YOUR_GITHUB_EMAIL"
!git config --global user.name "YOUR_GITHUB_USERNAME"
!git commit -m "Homework 5 implementation"
!git push origin main


## Submission

Run the following cell to bundle your submission (modify UTID accordingly).

If you notice that your bundle is too large, you can modify the `bundle.py` script and ignore large files by adding them manually to `BLACKLIST`.

After the bundler and grader run, right click and download your bundled `.zip` file from the Colab UI.


In [ ]:
# Make sure you're in the homework5 directory
%cd /content/{os.environ['REPO']}/homework5

# Replace YOUR_UT_ID with your actual UT ID
!python3 bundle.py homework YOUR_UT_ID

# Check the bundle was created
!ls -lh YOUR_UT_ID.zip

# Optional: run the grader with your bundled homework to double check
# !python3 -m grader YOUR_UT_ID.zip -vv --disable_color


In [ ]:
# Download the bundle
from google.colab import files
files.download(f"/content/{os.environ['REPO']}/homework5/YOUR_UT_ID.zip")


## Tensorboard (Optional)

Monitor your training progress with TensorBoard.


In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs
